In [ ]:
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/Iris.csv')
df.drop(columns=['Id'], inplace=True)
df['Species'] = df['Species'].map({'Iris-setosa':0,'Iris-versicolor':1,'Iris-virginica':2})
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
train, test = train_test_split(df, test_size=0.30, random_state=42)

In [ ]:
train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
81,5.5,2.4,3.7,1.0,1
133,6.3,2.8,5.1,1.5,2
137,6.4,3.1,5.5,1.8,2
75,6.6,3.0,4.4,1.4,1
109,7.2,3.6,6.1,2.5,2


Torch Tensors

In [ ]:
train_torch = torch.tensor(train.values, dtype=torch.float32)
test_torch = torch.tensor(test.values, dtype=torch.float32)
train_torch.dtype

torch.float32

Dataset

In [ ]:
class IrisClass(torch.utils.data.Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return self.data.size(0) # self.data.shape[0]

  def __getitem__(self, idx):
    return self.data[idx, :-1], self.data[idx, -1]

In [ ]:
train_dataset = IrisClass(train_torch)
test_dataset = IrisClass(test_torch)
len(test_dataset)

45

DataLoader

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=20)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=20)

Model Class

In [ ]:
class IrisClassifier(torch.nn.Module):
  def __init__(self, in_dims, out_dims):
    super(IrisClassifier, self).__init__()
    self.in_dims = in_dims
    self.out_dims = out_dims
    self.input_layer = torch.nn.Linear(self.in_dims, 64)
    self.hidden_layer = torch.nn.Linear(64, 32)
    self.output_layer = torch.nn.Linear(32, self.out_dims)
    self.relu = torch.nn.ReLU()

  def forward(self, x_batch):
    out = self.relu(self.input_layer(x_batch))
    out = self.relu(self.hidden_layer(out))
    out = self.output_layer(out)
    return out

Training Loop

In [ ]:
model = IrisClassifier(4, 3)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.005)

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
  for batch_x, batch_y in train_loader:
    model.train()
    #clear out the gradients from the last step loss.backward()
    optimizer.zero_grad()
    
    #forward feed
    output_train = model(batch_x)

    #calculate the loss
    loss_train = criterion(output_train, batch_y.long())
    
    #backward propagation: calculate gradients
    loss_train.backward()

    #update the weights
    optimizer.step()

    with torch.no_grad():
      output_test = model(test_torch[:, :-1])
      loss_test = criterion(output_test, test_torch[:, -1].long())
  
  if (epoch + 1) % 5 == 0:
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")


Epoch 5/20, Train Loss: 0.0063, Test Loss: 0.0611
Epoch 10/20, Train Loss: 0.0052, Test Loss: 0.0595
Epoch 15/20, Train Loss: 0.0045, Test Loss: 0.0578
Epoch 20/20, Train Loss: 0.0040, Test Loss: 0.0567


Evaluation / Prediction

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
model.eval()
with torch.no_grad():
  y_pred = np.argmax(model(test_torch[:, :-1]), 1)

In [ ]:
print(classification_report(test_torch[:, -1], y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        19
         1.0       1.00      1.00      1.00        13
         2.0       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

